In [ ]:
!kaggle datasets download -d uciml/sms-spam-collection-dataset
!unzip sms-spam-collection-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset
License(s): unknown
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 49.6MB/s]
Archive:  sms-spam-collection-dataset.zip
  inflating: spam.csv                


In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
print(pd.__version__)
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import re
import string

2.2.2


In [ ]:
data = pd.read_csv("spam.csv", encoding='latin-1')
data_copy = data.copy()
data.head()


FileNotFoundError: [Errno 2] No such file or directory: 'spam.csv'

In [ ]:
#remove the unnecessary columns
data = data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)
data.head()

KeyError: "['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'] not found in axis"

In [ ]:
#Changing the names of columns
data = data.rename(columns={'v1': 'label', 'v2': 'text'})

data.head()

#copy the data to data_copy2 to back up to original data if needed
data_copy2 = data.copy()


In [ ]:
data_copy2.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#turn spam/ham to numbers
data_copy2['label'] = data_copy2['label'].astype(str).str.strip().str.lower().map({'ham': 0, 'spam': 1})

data_copy2.head(25)

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
5,1,FreeMsg Hey there darling it's been 3 week's n...
6,0,Even my brother is not like to speak with me. ...
7,0,As per your request 'Melle Melle (Oru Minnamin...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...


In [ ]:
#clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    text = re.sub("\d+", "", text)
    text = re.sub('<.*?>+', '', text)
    return text

In [ ]:
#run the clean text function
data_copy2['text'] = data_copy2['text'].apply(clean_text)
data_copy2.head(25)

,label,text
0,0,go until jurong point crazy available only in ...
1,0,ok lar joking wif u oni
2,1,free entry in a wkly comp to win fa cup final...
3,0,u dun say so early hor u c already then say
4,0,nah i dont think he goes to usf he lives aroun...
5,1,freemsg hey there darling its been weeks now ...
6,0,even my brother is not like to speak with me t...
7,0,as per your request melle melle oru minnaminun...
8,1,winner as a valued network customer you have b...
9,1,had your mobile months or more u r entitled t...


In [ ]:
#split train and test data
x_train, x_test, y_train, y_test = train_test_split(data_copy2['text'], data_copy2['label'], test_size=0.2, random_state=42)

In [ ]:
#tfidf vectorization
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)


In [ ]:
#training
model = MultinomialNB()
model.fit(x_train_tfidf, y_train)
#prediction
y_pred = model.predict(x_test_tfidf)

In [ ]:
#evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred))
#testing
example = ["Congratulations! You won 100 dollar gift card!"]
example_tfidf = vectorizer.transform(example)
print("Spam (1) or not (0):", model.predict(example_tfidf)[0])

Accuracy: 0.95
              precision    recall  f1-score   support

           0       0.95      1.00      0.97       965
           1       1.00      0.64      0.78       150

    accuracy                           0.95      1115
   macro avg       0.97      0.82      0.88      1115
weighted avg       0.95      0.95      0.95      1115

Spam (1) or not (0): 1
